**AgroVision - CNN Model** 🌿

- We'll be using PlantVillage as our primary dataset 🖼️

- Transfer Learning with MobileNetV2 ⚡

- Adam's Optimizer [ Noise & Efficiency ] 💪

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'plantdisease' dataset.
Path to dataset files: /kaggle/input/plantdisease


**Spliting Data into [ Train & Validation ]** 📂

In [8]:
!pip install split-folders

In [9]:
import splitfolders
from pathlib import Path

# 1. Define the path to your *current* unsplit dataset
#    (This is the 'PlantVillage' folder from your screenshot)
INPUT_FOLDER = Path("/kaggle/input/plantdisease/PlantVillage")

# 2. Define where you want the *new* split dataset to be created
#    (We'll put it right where our training script expects it)
OUTPUT_FOLDER = Path("/content/plant_dataset")

print(f"Splitting files from {INPUT_FOLDER} into {OUTPUT_FOLDER}...")

# 3. Run the split
#    This will split your data into 70% training and 30% validation
#    (8,921 train vs 3,926 val is ~ 70/30)
splitfolders.ratio(
    INPUT_FOLDER,
    output=OUTPUT_FOLDER,
    seed=42,             # Gives a reproducible random split
    ratio=(.7, .3),      # 70% training, 30% validation
    group_prefix=None,   # No file grouping
    move=False           # Copies files instead of moving (safer)
)

print("Done splitting! New folders are ready.")
print(f"Your new training data is in: {OUTPUT_FOLDER / 'train'}")
print(f"Your new validation data is in: {OUTPUT_FOLDER / 'val'}")

Splitting files from /kaggle/input/plantdisease/PlantVillage into /content/plant_dataset...


Copying files: 20639 files [02:05, 164.51 files/s]

Done splitting! New folders are ready.
Your new training data is in: /content/plant_dataset/train
Your new validation data is in: /content/plant_dataset/val


**Importing our Libraries for Training** 💻

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import RandomFlip, RandomRotation



> Parameters Definition & Paths



In [11]:
# --- 1. Define Parameters ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_DIR = 'plant_dataset/train'
VAL_DIR = 'plant_dataset/val'

# --- 2. Load Datasets ---
# Keras will automatically label the data 0, 1, 2, 3 based on folder names
train_dataset = image_dataset_from_directory(
    TRAIN_DIR,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

validation_dataset = image_dataset_from_directory(
    VAL_DIR,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Get the class names (Keras finds these from the folder names)
class_names = train_dataset.class_names
print(f"Found class names: {class_names}")

NUM_CLASSES = len(class_names)
print(f"Successfully set number of classes to: {NUM_CLASSES}")


Found 14440 files belonging to 15 classes.
Found 6198 files belonging to 15 classes.
Found class names: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Successfully set number of classes to: 15




> Data Preparation & Augmentation



In [12]:
# --- 3. Create a Data Augmentation Layer ---
# This creates more training data by randomly flipping/rotating images
# It helps the model generalize better and prevents overfitting.
data_augmentation = tf.keras.Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.2),
])

# --- 4. Prepare Datasets for Performance ---
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, augment=False):
    # Apply the MobileNetV2 preprocessing
    ds = ds.map(lambda x, y: (tf.keras.applications.mobilenet_v2.preprocess_input(x), y),
                num_parallel_calls=AUTOTUNE)

    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=AUTOTUNE)

    # Use buffered prefetching for speed
    return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(train_dataset, augment=True)
val_ds = prepare(validation_dataset)




> Transfer Learning with MobileNetV2

- *This gives a better Recognition*



In [13]:
# --- 5. Build the Transfer Learning Model ---

# Load MobileNetV2, pre-trained on ImageNet, but without its final classifier layer
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False, # <-- IMPORTANT
    weights='imagenet'
)

# Freeze the base model. We don't want to re-train its weights.
base_model.trainable = False

# Create our new model on top
inputs = Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False) # Run in inference mode
x = GlobalAveragePooling2D()(x) # Flattens the output
x = Dropout(0.2)(x) # Regularization to prevent overfitting
# Our new classifier layer. NUM_CLASSES (4) is the number of folders we have.
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs, outputs)

# --- 6. Compile the Model ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy', # Use this for integer labels
    metrics=['accuracy']
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 19,215 (75.06 KB)

 Non-trainable params: 2,257,984 (8.61 MB)



> Model Training ⚙️



In [ ]:
# --- 7. Train the Model ---
print("\n--- Starting Model Training ---")

# This is where the model "learns" from your 8,921 images
history = model.fit(
    train_ds,
    epochs=10, # Start with 10, you can increase this later
    validation_data=val_ds
)

print("\n--- Training Complete ---")




> Model Evaluation ✅



In [ ]:
# --- 8. Evaluate and Save ---

# This will give you the FINAL statistics
print("\n--- Evaluating on Validation Set ---")
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Validation Loss: {loss:.4f}")

# Save your new, trained model!
model.save('AgroVision.h5')
print("\nModel saved as 'AgroVision.h5'")